In [ ]:
import utils
import os
import pandas as pd

In [ ]:
HP_PATH = 'data/hp.obo'

LABITEMS_PATH = 'data/D_LABITEMS.csv'
ANNOTATION_PATH = 'data/loinc2hpo-annotations.tsv'
LABEVENTS_PATH = 'data/LABEVENTS.csv'

LABEVENTS_HPO_PATH = 'data/OUT_LABEVENTS_HPO.csv'

SAMPLE_DIAGNOSES_ICD_HPO_PATH = 'data/DIAGNOSES_ICD_HPO.csv'
SAMPLE_LABEVENTS_HPO_PATH = 'data/LABEVENTS_HPO.csv'


In [ ]:
def download_if_not_already(path: str, url: str):
    if not os.path.isfile(path):
        import requests  # for downloading the hpo
        r = requests.get(url)
        with open(path, 'wb') as f:
            f.write(r.content)


In [ ]:
# download hp.obo if it is not already
download_if_not_already(HP_PATH, 'http://purl.obolibrary.org/obo/hp.obo')
download_if_not_already(ANNOTATION_PATH, 'https://raw.githubusercontent.com/TheJacksonLaboratory/loinc2hpoAnnotation/master/loinc2hpo-annotations.tsv')
download_if_not_already(LABITEMS_PATH, 'https://physionet.org/files/mimiciii-demo/1.4/D_LABITEMS.csv')
download_if_not_already(LABEVENTS_PATH, 'https://physionet.org/files/mimiciii-demo/1.4/LABEVENTS.csv')

In [ ]:
if not os.path.isfile(LABEVENTS_HPO_PATH):
    utils.add_hpo_information(
        LABITEMS_PATH, LABEVENTS_PATH,  ANNOTATION_PATH, LABEVENTS_HPO_PATH)


In [ ]:
hpo = utils.read_hpo_from_obo(HP_PATH)


In [ ]:
g = utils.make_graph_to_depth(hpo.entries_by_id['HP:0000118'], 1)
print(hpo.root.other_tags)
g

In [ ]:
labevents_df = pd.read_csv(SAMPLE_LABEVENTS_HPO_PATH)
diagnoses_df = pd.read_csv(SAMPLE_DIAGNOSES_ICD_HPO_PATH)

In [ ]:
labevents = set()
for line in labevents_df.active_hpo_features:
    if type(line) is str:
        for feature in line.split(';'):
            labevents.add (feature)
diagnoses = set()
for line in diagnoses_df.hpo_features:
    if type(line) is str:
        for feature in line.split(';'):
            diagnoses.add (feature)


In [ ]:
# show a graph to maybe help us understand how we can use the graph

g = utils.make_graph_2(hpo, labevents, diagnoses)
# g